In [1]:
import pandas as pd

In [213]:
orders = pd.DataFrame({'area':['a','a','a','a'], 'code': [17, 17,17,17], 'requires':[0,1,100,2]})
graphic = pd.DataFrame({'area': ['a'], 'code': [17], 
                        'date':['2022-06-11'], 
                        'volume':[5]})

In [193]:
orders = pd.read_excel('orders-test.xlsx')
graphic = pd.read_excel('graphic-test.xlsx')

In [212]:
graphic['date'] = pd.to_datetime(graphic['date'])

In [198]:
orders

,area,code,requires
0,a,17,0
1,a,17,1
2,a,17,100
3,a,17,2


In [199]:
graphic

,area,code,date,volume
0,a,17,2022-06-11,5


In [200]:
graphic = graphic.merge(orders, on=["area", "code"])

In [201]:
graphic

,area,code,date,volume,requires
0,a,17,2022-06-11,5,0
1,a,17,2022-06-11,5,1
2,a,17,2022-06-11,5,100
3,a,17,2022-06-11,5,2


In [202]:
graphic["volume_cumsum"] = graphic.groupby(["area", "code"])["volume"].cumsum()

In [203]:
graphic

,area,code,date,volume,requires,volume_cumsum
0,a,17,2022-06-11,5,0,5
1,a,17,2022-06-11,5,1,10
2,a,17,2022-06-11,5,100,15
3,a,17,2022-06-11,5,2,20


In [204]:
required_exceeded_mask = (graphic["volume_cumsum"] >= graphic["requires"]) & (graphic['requires'] != 0)

In [205]:
required_exceeded_mask

0    False
1     True
2    False
3     True
dtype: bool

In [206]:
graphic[required_exceeded_mask]

,area,code,date,volume,requires,volume_cumsum
1,a,17,2022-06-11,5,1,10
3,a,17,2022-06-11,5,2,20


In [207]:
min_exceeding_date = (
    graphic[required_exceeded_mask].groupby(["area", "code"])["date"].min()
)

In [208]:
min_exceeding_date

area  code
a     17     2022-06-11
Name: date, dtype: datetime64[ns]

In [209]:
result = orders.merge(
    min_exceeding_date, left_on=["area", "code"], right_index=True, how="outer"
)

In [210]:
result

,area,code,requires,date
0,a,17,0,2022-06-11
1,a,17,1,2022-06-11
2,a,17,100,2022-06-11
3,a,17,2,2022-06-11


In [190]:
result.loc[result['requires'] == 0] = 0

In [191]:
result

,area,code,requires,date
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,a,17,5,2022-06-11 00:00:00


In [ ]:
# Add the cummulative sum of volume (per group)
graphic["volume_cumsum"] = graphic.groupby(["area", "code"])["volume"].cumsum()

# Join with orders to add the requires column
graphic = graphic.merge(orders, on=["area", "code"])

# Create a mask to leave only rows where requires is exceeded by the cumsum
required_exceeded_mask = graphic["volume_cumsum"] >= graphic["requires"]

# The minimal date where the requires was exceeded
min_exceeding_date = (
    graphic[required_exceeded_mask].groupby(["area", "code"])["date"].min()
)

# Merge the minimal date to the orders dataframe - while keeping missing values
result = orders.merge(
    min_exceeding_date, left_on=["area", "code"], right_index=True, how="outer"
)

In [22]:
result

,area,code,requires,date
0,a,177,10.0,2022-06-13
1,b,23,3.0,NaN
2,c,10,6.0,2022-07-01


In [214]:
def data_shipment(city, code, required_to_make):
    # проверяем есть ли вообще в списке graphic
    if graphic[(graphic['area']==city) & (graphic['code']==code)].empty:
        return np.nan
    # если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        print(2)
        return np.nan
    # иначе если есть и объема в графике достаточно, то мы    
    
    elif graphic.loc[(graphic['area']==city) & (graphic['code']==code), 'volume'].iloc[0] >= required_to_make:   
        print('enough')
        # вычитаем из объема
        # добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['area']==city) & (graphic['code']==code) & 
                                    (graphic['date']==graphic.loc[(graphic['area']==city) 
                                    & (graphic['code']==code), 'date'].iloc[0]),'date'].iloc[0]
        graphic.loc[(graphic['area']==city) & (graphic['code']==code) & (graphic['date']==data_of_making),
                    'volume'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['area']==city) & (graphic['code']==code)]['date'].iloc[0]
        return data
    # иначе если недостаточно объема чем требуется ...
    else:
        # иначе если возвращается только одна дата, возвращает -
        #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'] = 0
        if len(graphic.loc[(graphic['area']==city) & (graphic['code']==code)]) == 1:
            print(3)
            return np.nan
        # и возращается несколько дат
        elif len(graphic.loc[(graphic['area']==city) & (graphic['code']==code)]) > 1:
            # вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['area']==city) & (graphic['code']==code)])
            # вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            # создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['area']==city) & (graphic['code']==code), 'volume'].iloc[0]
            list_of_dates_to_be_deleted = []

            # пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                len_of_dates-=1
                # переходим к следующей
                i+=1
                # и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['area']==city) & (graphic['code']==code), 'volume'].iloc[i]
                list_of_dates_to_be_deleted.append(graphic.loc[(graphic['area']==city) & (graphic['code']==code) & 
                                             (graphic['date']==graphic.loc[(graphic['area']==city) 
                                            & (graphic['code']==code), 'date'].iloc[i-1]),
                                             'date'].iloc[0])                    
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                return np.nan
            # иначе
            else:
                #нужно обнулить или удалить предыдущие даты, которые схлопнулись
                for j in (list_of_dates_to_be_deleted):
                    graphic.drop(graphic[(graphic['area']==city) & (graphic['code']==code) & 
                           (graphic['date']==j)].index, inplace=True)
                    i-=1
                data_of_making = graphic.loc[(graphic['area']==city) & (graphic['code']==code) & 
                                             (graphic['date']==graphic.loc[(graphic['area']==city) 
                                            & (graphic['code']==code), 'date'].iloc[i]),
                                             'date'].iloc[0]
        
            # вычитаем
            graphic.loc[(graphic['area']==city) & (graphic['code']==code) & 
                        (graphic['date']==data_of_making), 'volume'] = storage_var-required_to_make
 
            data = graphic[(graphic['area']==city) & (graphic['code']==code)]['date'].iloc[i] 
            return data

In [ ]:
orders['date'] = orders.apply(lambda x: data_shipment(x['area'], x['code'], x['requires']), axis=1)

In [123]:
orders

,area,code,requires,date
0,a,177,10.0,2022-06-13
1,b,23,3.0,NaN
2,c,10,6.0,2022-07-01


In [124]:
graphic

,area,code,date,volume
2,a,177,2022-06-13,0.0
3,c,10,2022-07-01,0.0
4,c,10,2022-07-03,3.0
